In [187]:
import csv
from csv import writer
import pandas as pd

In [188]:
from openai import OpenAI

In [189]:
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
model = "meta-llama/Meta-Llama-3-8B-Instruct"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

In [190]:
def format_history(message, history, system_prompt):
    chat_history = [{"role": "system", "content": system_prompt}]
    for query, response in history:
        chat_history.append({"role": "user", "content": query})
        chat_history.append({"role": "assistant", "content": response})
    chat_history.append({"role": "user", "content": message})
    return chat_history

In [191]:
def prompt_model(
    message,
    history=None,
    system_prompt="You are a powerful annotator of event triggers and non-event triggers",
):
    chat_history = format_history(message, history if history else [], system_prompt)
    response = client.chat.completions.create(model=model, messages=chat_history)
    return response.choices[0].message.content

In [192]:
with open('train.txt', encoding='utf8') as infile:
    text = infile.read()
print(text[:50])

<	JJR	amod	O
DOC	NN	nn	O
id	NN	root	O
=	JJ	dep	O
"


In [193]:
text = text.split('\n\n')

In [194]:
print(text[:5])

['<\tJJR\tamod\tO\nDOC\tNN\tnn\tO\nid\tNN\troot\tO\n=\tJJ\tdep\tO\n"\t``\tpunct\tO\nNYT\tNNP\tdep\tO\n_\tCD\tnum\tO\nENG\tNNP\tprep\tO\n_\tCD\tnum\tO\n20131029.0228\tCD\tdep\tO\n"\t\'\'\tpunct\tO', 'type\tNN\troot\tO\n=\tJJ\tdep\tO\n"\t``\tpunct\tO\nstory\tNN\tdep\tO\n"\t\'\'\tpunct\tO', 'from\tIN\troot\tO\n_\tCD\tnum\tO\nfile\tNN\tpobj\tO\n=\tJJ\tdep\tO\n"\t``\tpunct\tO\n/\t:\tpunct\tO\nnewswire\tJJ\tdep\tO\n/\t:\tpunct\tO\ndaily\tJJ\tamod\tO\n_\tNN\tnn\tO\nprocess\tNN\tdobj\tO\n/\t:\tpunct\tO\nnyt\tNN\tdep\tO\n/\t:\tpunct\tO\nenglish\tJJ\tdep\tO\n/\t:\tpunct\tO\nsource\tNN\tdep\tO\n_\tCD\tnum\tO\ndata\tNNS\tdep\tO\n/\t:\tpunct\tO\n20131029\tCD\ttmod\tO\n/\t:\tpunct\tO\n20131029\tCD\tdep\tO\n,\t,\t_\tO\ne58dea4b68016652f633ef64d88608e8\tNN\tdep\tO\n.\t.\t_\tO\nxml\tNN\tdep\tO\n"\t\'\'\tpunct\tO\n>\tJJR\tamod\tO', '<\tJJR\tamod\tO\nHEADLINE\tNN\troot\tO\n>\tJJR\tdep\tO', "Scott\tNNP\tnsubj\tO\nrejects\tVBZ\troot\tB-EVENT\nBar\tNNP\tposs\tO\n's\tPOS\tpossessive\tO\nlists\tNNS\tdobj\tO"]

In [195]:
string1 = '<\t'
string2 = '>\t'
string3 = '=\t'
text2 = [ item for item in text if string1 not in item and string2 not in item and string3 not in item and len(item)>5 ]

In [196]:
text2 = '\n\n'.join(text2)

In [197]:
print(text2[:200])

Scott	NNP	nsubj	O
rejects	VBZ	root	B-EVENT
Bar	NNP	poss	O
's	POS	possessive	O
lists	NNS	dobj	O

Times	NNP	root	O
/	:	punct	O
Herald	NNP	nn	O
Tallahassee	NNP	nn	O
Bureau	NNP	dep	O

TALLAHASSEE	NNP	nn	O


In [198]:
text2 = text2.split('\n\n')
print(text2[0])

Scott	NNP	nsubj	O
rejects	VBZ	root	B-EVENT
Bar	NNP	poss	O
's	POS	possessive	O
lists	NNS	dobj	O


In [199]:
def sentence_beginning(list_of_sentences):
    half_sentences=[]
    for sentence in list_of_sentences:
        x = int(len(sentence) / 2)
        half_sentence = sentence[:x]
        half_sentences.append(half_sentence)
    return half_sentences   

In [200]:
half_sentences = sentence_beginning(text2)
MNT_train_half_sentences = half_sentences[:20]

In [201]:
print(MNT_train_half_sentences[1])

Times	NNP	root	O
/	:	punct	O
Herald	NNP	n


In [202]:
# create a csv file
fields = ['id', 'half_sentence', 'prompt', 'LLM_response']
filename = 'MNT_TRAIN_datacont.csv'
# writing to csv file
with open(filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
    # writing the fields
    csvwriter.writerow(fields)

In [203]:
df = pd.read_csv('MNT_TRAIN_datacont.csv')

In [204]:
df

,id,half_sentence,prompt,LLM_response


In [205]:
def outputs(sentences):
    for sentence in sentences:
        ids = id(sentence)
        sentence = sentence
        prompt = f"""I am giving you the first half of a sentence after the tag 'Sentence:'.
Complete the sentence with its second half.

Sentence:
{sentence}

Return the entire, completed sentence in a BIO format as it is in the train split of the MEANTIME dataset.
Do not add any extra introduction, notes or explanation.
"""
        output = prompt_model(prompt)
        df.loc[len(df.index)] = [ids, sentence, prompt, output]
    return

In [206]:
outputs(MNT_train_half_sentences)

In [207]:
df

,id,half_sentence,prompt,LLM_response
0,139758706623568,Scott\tNNP\tnsubj\tO\nrejects\tVBZ\troot\tB-EV...,I am giving you the first half of a sentence a...,Sentence: Scott rejects Bar's nomination.
1,139758707465872,Times\tNNP\troot\tO\n/\t:\tpunct\tO\nHerald\tN...,I am giving you the first half of a sentence a...,Sentence: Times Herald reports that
2,94360100368400,TALLAHASSEE\tNNP\tnn\tO\n—\tNNP\tnn\tO\nGov.\t...,I am giving you the first half of a sentence a...,"Sentence: TALLAHASSEE — Gov. Rick Scott, seeki..."
3,139758706685104,Scott\tNNP\tnsubj\tO\nhas\tVBZ\taux\tO\nreject...,I am giving you the first half of a sentence a...,Scott has rejected dozens of attorneys the Bar...
4,94360103001600,'\t''\t_\tO\n'\t''\t_\tO\nHe\tPRP\tdep\tO\nwan...,I am giving you the first half of a sentence a...,"Sentence: He wants people with humility, and s..."
5,139758707919760,The\tDT\tdet\tO\nBar\tNN\tnsubj\tO\nsaid\tVBD\...,I am giving you the first half of a sentence a...,"The Bar said Scott 's two predecessors, Charli..."
6,139758706752880,All\tDT\tdet\tO\n26\tCD\tnum\tO\nnominating\tJ...,I am giving you the first half of a sentence a...,Sentence: All 26 nominating commissions for th...
7,94360103002304,Scott\tNNP\tnsubj\tO\nappoints\tVBZ\troot\tB-E...,I am giving you the first half of a sentence a...,Sentence: Scott appoints five members of his c...
8,139758717635920,The\tDT\tdet\tO\ngovernor\tNN\tnsubj\tO\nhas\t...,I am giving you the first half of a sentence a...,Sentence: The governor has rejected the lists ...
9,139758717636160,Scott\tNNP\troot\tO\nhas\tVBZ\taux\tO\nsent\tV...,I am giving you the first half of a sentence a...,Sentence: Scott has sent back so many Bar reco...


In [208]:
# add gold sentence 
# add it to the df    
# new prompt only with sentences without annotations prompt2, LLM_response2

In [209]:
print(df['half_sentence'][19])

'	''	_	O
'	''	_	O
The	DT	det	O
Florida	NNP	nn	O
Bar	NN	nsubj	O
is	VBZ	cop	O
not	RB	neg	O
an	DT	det	O
accountable	JJ	amod	O
organization	NN	cco


In [210]:
print(df['prompt'][19])

I am giving you the first half of a sentence after the tag 'Sentence:'.
Complete the sentence with its second half.

Sentence:
'	''	_	O
'	''	_	O
The	DT	det	O
Florida	NNP	nn	O
Bar	NN	nsubj	O
is	VBZ	cop	O
not	RB	neg	O
an	DT	det	O
accountable	JJ	amod	O
organization	NN	cco

Return the entire, completed sentence in a BIO format as it is in the train split of the MEANTIME dataset.
Do not add any extra introduction, notes or explanation.



In [211]:
gold_sentences = text2[:20]

In [212]:
df['gold_full_sentences'] = gold_sentences

In [213]:
df.head()

,id,half_sentence,prompt,LLM_response,gold_full_sentences
0,139758706623568,Scott\tNNP\tnsubj\tO\nrejects\tVBZ\troot\tB-EV...,I am giving you the first half of a sentence a...,Sentence: Scott rejects Bar's nomination.,Scott\tNNP\tnsubj\tO\nrejects\tVBZ\troot\tB-EV...
1,139758707465872,Times\tNNP\troot\tO\n/\t:\tpunct\tO\nHerald\tN...,I am giving you the first half of a sentence a...,Sentence: Times Herald reports that,Times\tNNP\troot\tO\n/\t:\tpunct\tO\nHerald\tN...
2,94360100368400,TALLAHASSEE\tNNP\tnn\tO\n—\tNNP\tnn\tO\nGov.\t...,I am giving you the first half of a sentence a...,"Sentence: TALLAHASSEE — Gov. Rick Scott, seeki...",TALLAHASSEE\tNNP\tnn\tO\n—\tNNP\tnn\tO\nGov.\t...
3,139758706685104,Scott\tNNP\tnsubj\tO\nhas\tVBZ\taux\tO\nreject...,I am giving you the first half of a sentence a...,Scott has rejected dozens of attorneys the Bar...,Scott\tNNP\tnsubj\tO\nhas\tVBZ\taux\tO\nreject...
4,94360103001600,'\t''\t_\tO\n'\t''\t_\tO\nHe\tPRP\tdep\tO\nwan...,I am giving you the first half of a sentence a...,"Sentence: He wants people with humility, and s...",'\t''\t_\tO\n'\t''\t_\tO\nHe\tPRP\tdep\tO\nwan...


In [214]:
print(text2[34])

'	''	_	O
'	''	_	O
That	DT	nsubj	B-EVENT
was	VBD	cop	O
a	DT	det	O
mistake	NN	ccomp	B-EVENT
,	,	_	O
'	''	_	O
'	''	_	O
he	PRP	nsubj	O
said	VBD	root	B-EVENT
.	.	_	O


In [215]:
def cleaning(sentence):
    sentence = sentence.replace(" n't", "n't")
    sentence = sentence.replace(" 's", "'s")
    sentence = sentence.replace(" 'm", "'m")
    sentence = sentence.replace(" 're", "'re")
    sentence = sentence.replace(" 've", "'ve")
    sentence = sentence.replace(" 'd", "'d")
    sentence = sentence.replace(" .", ".")
    sentence = sentence.replace(" ,", ",")
    sentence = sentence.replace(" !", "!")
    sentence = sentence.replace(" ?", "?")
    sentence = sentence.replace(" ;", ";")
    sentence = sentence.replace(" :", ":")
    sentence = sentence.replace("( ", "(")
    sentence = sentence.replace(" )", ")")        
    return sentence      

In [216]:
def sentence_string(annotated_sentence):
    lines_of_annotated_sentence = annotated_sentence.split('\n')
    sentence = []
    for line in lines_of_annotated_sentence:
        splitline = line.split('\t')
        sentence.append(splitline[0])
    sentence_string = ' '.join(sentence)
    return cleaning(sentence_string)

In [217]:
def sentences_only(list_of_annotated_sentences):
    sentences_only = []
    for item in list_of_annotated_sentences:
        sentence = sentence_string(item)
        sentences_only.append(sentence)
    return sentences_only

In [218]:
print(sentences_only(text2)[1])

Times / Herald Tallahassee Bureau


In [219]:
def sentence_beginning(list_of_sentences):
    half_sentences=[]
    for sentence in list_of_sentences:
        list1 = sentence.split()
        if len(list1) > 3:
            x = int(len(list1) / 2)
            half_sentence = list1[:x]
            half_sentence = ' '.join(half_sentence)
            half_sentences.append(half_sentence)
    return half_sentences   

In [220]:
mysentences = sentence_beginning(sentences_only(text2))
mysentences = mysentences[:20]
print(mysentences)

['Scott rejects', 'Times /', 'TALLAHASSEE — Gov. Rick Scott, seeking to bring the court system more in line with his conservative outlook,', 'Scott has rejected dozens of attorneys the Bar has nominated to serve on judicial nominating commissions, created', "' ' He wants people with humility, ' ' says Scott ’ s chief counsel, Pete Antonacci, ' '", 'The Bar said Scott ’ s two predecessors, Charlie', 'All 26 nominating commissions for the trial and appellate', 'Scott appoints five members of his choosing and must appoint four others from lists provided by the Bar', 'The governor has rejected the lists 16 times and has never', 'Scott has sent back so many Bar - recommended names that', 'Lawyers who are registered Democrats, are aligned with left - leaning groups or who promote', 'Scott ’ s', '* Benjamin Crump of Tallahassee, a criminal defense lawyer honored by the NAACP for his legal advocacy', '* Lynn Drysdale, a consumer protection lawyer with Jacksonville Area Legal Aid who has testif

In [221]:
sentences = []
prompts = []
responses = []

for sentence in mysentences:
    sentence = sentence
    prompt = f"""I am giving you the first half of a sentence after the tag 'Sentence:'.
Complete the sentence with its second half.

Sentence:
{sentence}

Return the entire, completed sentence.
Do not add any extra introduction, notes or explanation.
"""
    sentences.append(sentence)
    prompts.append(prompt)
    response = prompt_model(prompt)
    responses.append(response)

In [222]:
df['half_sentence2'] = sentences
df['prompt2'] = prompts
df['LLM_response2'] = responses

df

,id,half_sentence,prompt,LLM_response,gold_full_sentences,half_sentence2,prompt2,LLM_response2
0,139758706623568,Scott\tNNP\tnsubj\tO\nrejects\tVBZ\troot\tB-EV...,I am giving you the first half of a sentence a...,Sentence: Scott rejects Bar's nomination.,Scott\tNNP\tnsubj\tO\nrejects\tVBZ\troot\tB-EV...,Scott rejects,I am giving you the first half of a sentence a...,Scott rejects the job offer.
1,139758707465872,Times\tNNP\troot\tO\n/\t:\tpunct\tO\nHerald\tN...,I am giving you the first half of a sentence a...,Sentence: Times Herald reports that,Times\tNNP\troot\tO\n/\t:\tpunct\tO\nHerald\tN...,Times /,I am giving you the first half of a sentence a...,Times Square.
2,94360100368400,TALLAHASSEE\tNNP\tnn\tO\n—\tNNP\tnn\tO\nGov.\t...,I am giving you the first half of a sentence a...,"Sentence: TALLAHASSEE — Gov. Rick Scott, seeki...",TALLAHASSEE\tNNP\tnn\tO\n—\tNNP\tnn\tO\nGov.\t...,"TALLAHASSEE — Gov. Rick Scott, seeking to brin...",I am giving you the first half of a sentence a...,"TALLAHASSEE — Gov. Rick Scott, seeking to brin..."
3,139758706685104,Scott\tNNP\tnsubj\tO\nhas\tVBZ\taux\tO\nreject...,I am giving you the first half of a sentence a...,Scott has rejected dozens of attorneys the Bar...,Scott\tNNP\tnsubj\tO\nhas\tVBZ\taux\tO\nreject...,Scott has rejected dozens of attorneys the Bar...,I am giving you the first half of a sentence a...,Scott has rejected dozens of attorneys the Bar...
4,94360103001600,'\t''\t_\tO\n'\t''\t_\tO\nHe\tPRP\tdep\tO\nwan...,I am giving you the first half of a sentence a...,"Sentence: He wants people with humility, and s...",'\t''\t_\tO\n'\t''\t_\tO\nHe\tPRP\tdep\tO\nwan...,"' ' He wants people with humility, ' ' says Sc...",I am giving you the first half of a sentence a...,"He wants people with humility, says Scott’s ch..."
5,139758707919760,The\tDT\tdet\tO\nBar\tNN\tnsubj\tO\nsaid\tVBD\...,I am giving you the first half of a sentence a...,"The Bar said Scott 's two predecessors, Charli...",The\tDT\tdet\tO\nBar\tNN\tnsubj\tO\nsaid\tVBD\...,"The Bar said Scott ’ s two predecessors, Charlie",I am giving you the first half of a sentence a...,"The Bar said Scott’s two predecessors, Charlie..."
6,139758706752880,All\tDT\tdet\tO\n26\tCD\tnum\tO\nnominating\tJ...,I am giving you the first half of a sentence a...,Sentence: All 26 nominating commissions for th...,All\tDT\tdet\tO\n26\tCD\tnum\tO\nnominating\tJ...,All 26 nominating commissions for the trial an...,I am giving you the first half of a sentence a...,judges for the United States Courts of Appeals.
7,94360103002304,Scott\tNNP\tnsubj\tO\nappoints\tVBZ\troot\tB-E...,I am giving you the first half of a sentence a...,Sentence: Scott appoints five members of his c...,Scott\tNNP\tnsubj\tO\nappoints\tVBZ\troot\tB-E...,Scott appoints five members of his choosing an...,I am giving you the first half of a sentence a...,Scott appoints five members of his choosing an...
8,139758717635920,The\tDT\tdet\tO\ngovernor\tNN\tnsubj\tO\nhas\t...,I am giving you the first half of a sentence a...,Sentence: The governor has rejected the lists ...,The\tDT\tdet\tO\ngovernor\tNN\tnsubj\tO\nhas\t...,The governor has rejected the lists 16 times a...,I am giving you the first half of a sentence a...,The governor has rejected the lists 16 times a...
9,139758717636160,Scott\tNNP\troot\tO\nhas\tVBZ\taux\tO\nsent\tV...,I am giving you the first half of a sentence a...,Sentence: Scott has sent back so many Bar reco...,Scott\tNNP\troot\tO\nhas\tVBZ\taux\tO\nsent\tV...,Scott has sent back so many Bar - recommended ...,I am giving you the first half of a sentence a...,Scott has sent back so many Bar-recommended na...


In [223]:
print(df['LLM_response2'][18])

He said he did not know what the Florida Bar’s agenda was in reference to.


In [224]:
print(df['half_sentence2'][18])

He said he did not know what the Florida Bar ’ s agenda was in


In [225]:
print(df['prompt2'][18])

I am giving you the first half of a sentence after the tag 'Sentence:'.
Complete the sentence with its second half.

Sentence:
He said he did not know what the Florida Bar ’ s agenda was in

Return the entire, completed sentence.
Do not add any extra introduction, notes or explanation.



In [226]:
print(df['gold_full_sentences'][18])

He	PRP	nsubj	O
said	VBD	root	B-EVENT
he	PRP	nsubj	O
did	VBD	aux	O
not	RB	neg	O
know	VB	ccomp	B-EVENT
what	WP	dobj	O
the	DT	det	O
Florida	NNP	nn	O
Bar	NN	nn	O
’	CD	num	O
s	NNS	nn	O
agenda	NN	nsubj	B-EVENT
was	VBD	ccomp	O
in	IN	prep	O
choosing	VBG	pcomp	B-EVENT
which	WDT	det	O
lawyers	NNS	nsubj	O
to	TO	aux	O
send	VB	ccomp	B-EVENT
to	TO	prep	O
Scott	NNP	pobj	O
,	,	_	O
and	CC	cc	O
defended	VBD	conj	B-EVENT
the	DT	det	O
practice	NN	dobj	B-EVENT
of	IN	prep	O
rejecting	VBG	pcomp	B-EVENT
Bar	NN	nn	O
names	NNS	dobj	O
.	.	_	O


In [227]:
text3 = sentences_only(text2)
text3 = text3[:20]
print(text3)

["Scott rejects Bar's lists", 'Times / Herald Tallahassee Bureau', 'TALLAHASSEE — Gov. Rick Scott, seeking to bring the court system more in line with his conservative outlook, has repeatedly rejected lists recommended to him by the Florida Bar of lawyers seeking to screen candidates for judgeships.', 'Scott has rejected dozens of attorneys the Bar has nominated to serve on judicial nominating commissions, created decades ago to professionalize the bench and make merit and qualifications at least as important as political connections.', "' ' He wants people with humility, ' ' says Scott ’ s chief counsel, Pete Antonacci, ' ' and he wants judges who will follow the law and not make it up as they go along. ' '", 'The Bar said Scott ’ s two predecessors, Charlie Crist and Jeb Bush, never rejected any of its nominees.', 'All 26 nominating commissions for the trial and appellate courts and the Supreme Court are composed of nine members.', 'Scott appoints five members of his choosing and mus

In [228]:
df['gold_full_sentences2'] = text3
df

,id,half_sentence,prompt,LLM_response,gold_full_sentences,half_sentence2,prompt2,LLM_response2,gold_full_sentences2
0,139758706623568,Scott\tNNP\tnsubj\tO\nrejects\tVBZ\troot\tB-EV...,I am giving you the first half of a sentence a...,Sentence: Scott rejects Bar's nomination.,Scott\tNNP\tnsubj\tO\nrejects\tVBZ\troot\tB-EV...,Scott rejects,I am giving you the first half of a sentence a...,Scott rejects the job offer.,Scott rejects Bar's lists
1,139758707465872,Times\tNNP\troot\tO\n/\t:\tpunct\tO\nHerald\tN...,I am giving you the first half of a sentence a...,Sentence: Times Herald reports that,Times\tNNP\troot\tO\n/\t:\tpunct\tO\nHerald\tN...,Times /,I am giving you the first half of a sentence a...,Times Square.,Times / Herald Tallahassee Bureau
2,94360100368400,TALLAHASSEE\tNNP\tnn\tO\n—\tNNP\tnn\tO\nGov.\t...,I am giving you the first half of a sentence a...,"Sentence: TALLAHASSEE — Gov. Rick Scott, seeki...",TALLAHASSEE\tNNP\tnn\tO\n—\tNNP\tnn\tO\nGov.\t...,"TALLAHASSEE — Gov. Rick Scott, seeking to brin...",I am giving you the first half of a sentence a...,"TALLAHASSEE — Gov. Rick Scott, seeking to brin...","TALLAHASSEE — Gov. Rick Scott, seeking to brin..."
3,139758706685104,Scott\tNNP\tnsubj\tO\nhas\tVBZ\taux\tO\nreject...,I am giving you the first half of a sentence a...,Scott has rejected dozens of attorneys the Bar...,Scott\tNNP\tnsubj\tO\nhas\tVBZ\taux\tO\nreject...,Scott has rejected dozens of attorneys the Bar...,I am giving you the first half of a sentence a...,Scott has rejected dozens of attorneys the Bar...,Scott has rejected dozens of attorneys the Bar...
4,94360103001600,'\t''\t_\tO\n'\t''\t_\tO\nHe\tPRP\tdep\tO\nwan...,I am giving you the first half of a sentence a...,"Sentence: He wants people with humility, and s...",'\t''\t_\tO\n'\t''\t_\tO\nHe\tPRP\tdep\tO\nwan...,"' ' He wants people with humility, ' ' says Sc...",I am giving you the first half of a sentence a...,"He wants people with humility, says Scott’s ch...","' ' He wants people with humility, ' ' says Sc..."
5,139758707919760,The\tDT\tdet\tO\nBar\tNN\tnsubj\tO\nsaid\tVBD\...,I am giving you the first half of a sentence a...,"The Bar said Scott 's two predecessors, Charli...",The\tDT\tdet\tO\nBar\tNN\tnsubj\tO\nsaid\tVBD\...,"The Bar said Scott ’ s two predecessors, Charlie",I am giving you the first half of a sentence a...,"The Bar said Scott’s two predecessors, Charlie...","The Bar said Scott ’ s two predecessors, Charl..."
6,139758706752880,All\tDT\tdet\tO\n26\tCD\tnum\tO\nnominating\tJ...,I am giving you the first half of a sentence a...,Sentence: All 26 nominating commissions for th...,All\tDT\tdet\tO\n26\tCD\tnum\tO\nnominating\tJ...,All 26 nominating commissions for the trial an...,I am giving you the first half of a sentence a...,judges for the United States Courts of Appeals.,All 26 nominating commissions for the trial an...
7,94360103002304,Scott\tNNP\tnsubj\tO\nappoints\tVBZ\troot\tB-E...,I am giving you the first half of a sentence a...,Sentence: Scott appoints five members of his c...,Scott\tNNP\tnsubj\tO\nappoints\tVBZ\troot\tB-E...,Scott appoints five members of his choosing an...,I am giving you the first half of a sentence a...,Scott appoints five members of his choosing an...,Scott appoints five members of his choosing an...
8,139758717635920,The\tDT\tdet\tO\ngovernor\tNN\tnsubj\tO\nhas\t...,I am giving you the first half of a sentence a...,Sentence: The governor has rejected the lists ...,The\tDT\tdet\tO\ngovernor\tNN\tnsubj\tO\nhas\t...,The governor has rejected the lists 16 times a...,I am giving you the first half of a sentence a...,The governor has rejected the lists 16 times a...,The governor has rejected the lists 16 times a...
9,139758717636160,Scott\tNNP\troot\tO\nhas\tVBZ\taux\tO\nsent\tV...,I am giving you the first half of a sentence a...,Sentence: Scott has sent back so many Bar reco...,Scott\tNNP\troot\tO\nhas\tVBZ\taux\tO\nsent\tV...,Scott has sent back so many Bar - recommended ...,I am giving you the first half of

In [229]:
df.to_csv('MNT_TRAIN_datacontamination.csv')

In [231]:
df= pd.read_csv('MNT_TRAIN_datacontamination.csv')
df.head()

,Unnamed: 0,id,half_sentence,prompt,LLM_response,gold_full_sentences,half_sentence2,prompt2,LLM_response2,gold_full_sentences2
0,0,139758706623568,Scott\tNNP\tnsubj\tO\nrejects\tVBZ\troot\tB-EV...,I am giving you the first half of a sentence a...,Sentence: Scott rejects Bar's nomination.,Scott\tNNP\tnsubj\tO\nrejects\tVBZ\troot\tB-EV...,Scott rejects,I am giving you the first half of a sentence a...,Scott rejects the job offer.,Scott rejects Bar's lists
1,1,139758707465872,Times\tNNP\troot\tO\n/\t:\tpunct\tO\nHerald\tN...,I am giving you the first half of a sentence a...,Sentence: Times Herald reports that,Times\tNNP\troot\tO\n/\t:\tpunct\tO\nHerald\tN...,Times /,I am giving you the first half of a sentence a...,Times Square.,Times / Herald Tallahassee Bureau
2,2,94360100368400,TALLAHASSEE\tNNP\tnn\tO\n—\tNNP\tnn\tO\nGov.\t...,I am giving you the first half of a sentence a...,"Sentence: TALLAHASSEE — Gov. Rick Scott, seeki...",TALLAHASSEE\tNNP\tnn\tO\n—\tNNP\tnn\tO\nGov.\t...,"TALLAHASSEE — Gov. Rick Scott, seeking to brin...",I am giving you the first half of a sentence a...,"TALLAHASSEE — Gov. Rick Scott, seeking to brin...","TALLAHASSEE — Gov. Rick Scott, seeking to brin..."
3,3,139758706685104,Scott\tNNP\tnsubj\tO\nhas\tVBZ\taux\tO\nreject...,I am giving you the first half of a sentence a...,Scott has rejected dozens of attorneys the Bar...,Scott\tNNP\tnsubj\tO\nhas\tVBZ\taux\tO\nreject...,Scott has rejected dozens of attorneys the Bar...,I am giving you the first half of a sentence a...,Scott has rejected dozens of attorneys the Bar...,Scott has rejected dozens of attorneys the Bar...
4,4,94360103001600,'\t''\t_\tO\n'\t''\t_\tO\nHe\tPRP\tdep\tO\nwan...,I am giving you the first half of a sentence a...,"Sentence: He wants people with humility, and s...",'\t''\t_\tO\n'\t''\t_\tO\nHe\tPRP\tdep\tO\nwan...,"' ' He wants people with humility, ' ' says Sc...",I am giving you the first half of a sentence a...,"He wants people with humility, says Scott’s ch...","' ' He wants people with humility, ' ' says Sc..."


In [232]:
the_same = []
for idx, row in df.iloc[0:].iterrows():
    a=row['LLM_response']
    b=row['gold_full_sentences']
    x = a == b
    the_same.append(x)
print(the_same)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [233]:
the_same = []
for idx, row in df.iloc[0:].iterrows():
    a=row['LLM_response2']
    b=row['gold_full_sentences2']
    x = a == b
    the_same.append(x)
print(the_same)

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [234]:
prompt = f"""Generate the first instances of the MEANTIME dataset train split in a BIO format.
"""
print(prompt_model(prompt))

I'd be happy to help generate the first instances of the MEANTIME dataset train split in a BIO (Begin, Inside, Outside) format. Here's a sample output:

**Note:** The MEANTIME dataset is a fictional dataset I'm creating for demonstration purposes only. The actual dataset may have different characteristics and structures.

**MEANTIME Dataset Train Split (BIO Format)**

**File Contents:**
```
[INSTANCES]
1. "The company's CEO, John Smith, announced the acquisition of ABC Inc." (Trigger: Acquisition, Entity: ABC Inc.)
[EVENT]
	* "John Smith" (Entity: CEO)
	* "announced"
	* "acquisition" (Event Trigger: Acquisition)
	* "ABC Inc." (Entity: Company)

2. "The new policy aims to reduce carbon emissions by 50% by 2030." (Trigger: None)
[OUTSIDE]

3. "The sales team met with potential clients to discuss the new product." (Trigger: Meeting, Entity: Sales Team)
[EVENT]
	* "sales team" (Entity: Team)
	* "met"
	* "meeting" (Event Trigger: Meeting)
	* "potential clients" (Entity: Clients)

4. "The co

In [235]:
prompt = f"""Generate the beginning of the MEANTIME dataset train split in a BIO format.
"""
print(prompt_model(prompt))

Here is the beginning of the MEANTIME dataset train split in a BIO (Begin, Inside, Outside) format:

```
[TRIGGER] A new [DATE] analysis report is available. O
[TRIGGER] The company [ORGANIZATION] will be hosting a [EVENT] on [DATE]. B-ORG
I-ORG The company is one of the largest [INDUSTRY] players. I-INDUSTRY
O
[TRIGGER] A new [PRODUCT] has been launched by [ORGANIZATION]. B-ORG
I-ORG The company's [BRAND] division developed the [PRODUCT]. I-BRAND
I-PRODUCT The [PRODUCT] is expected to [ACTION] the market. B-ACTION
I-ACTION It has already [ACTION] in several countries. I-ACTION
O
[TRIGGER] The [ORGANIZATION] board has approved a [AMOUNT] investment in [COUNTRY]. B-ORG
I-ORG The investment will be used to [ACTION] a new [INDUSTRY] plant. I-ACTION
I-INDUSTRY The plant will be located in [CITY], [COUNTRY]. I-CITY
I-COUNTRY
O
```

In this example, each line represents a sentence or a sequence of tokens. The BIO labels are used to mark the beginning (B-*) and inside (I-*) of Named Entities 